## AuScope-Cat Search and Download Tutorial
https://pypi.org/project/auscopecat/

The AuScope-Cat library has been created to facilitate easy programmatic searching of the AuScope Portal catalogue of data, and allow for easy easy downloads of available data. The following tutorial will step you through the basic functions.

## Installation
You can install the AuScope-Cat library from pypi.org as follows:
```
pip install auscopecat
```

In [ ]:
! pip install auscopecat

## Searching
### Searching for Downloadable Data
To search for downloadable data, use the `search` function in the `api` module.
```
from auscopecat.api import search
```
The `search` function will search the AuScope Portal catalogue and return results that will facilitate the downloading of data with the `download` function (more on that later).
```
search(pattern: str, ogc_types: list[ServiceType] = None,
       spatial_search_type: SpatialSearchType = None,
       bbox: dict = None) -> list[SimpleNamespace]
```
##### Parameters

| Name | Type | Description | Example | Required |
| ------ | ------ | ------ | ------ | ------ |
| pattern | string | The search string. If there are multiple words, any result that contains any one of the words will be considered a match. To match an exact series of words, use quotation marks, e.g. "Mt Phillips". | 'magnetite' | Yes |
| ogc_types | List[ServiceType] | A list of auscope_types.ServiceType that define OGC service types of interest. Only results containing one or more of these services will be returned. | [ServiceType.WMS, ServiceType.WFS] | No |
| spatial_search | SpatialSearchType | The type of spatial search to be performed with the supplied bounding box. More on spatial searching later. | SpatialSearch.INTERSECTS | No |
| bbox | A dict containing 'north', 'east', 'south' and 'west' values | The bounding box to be used when performing a spatial search. The bounding box will only be applied if a spatial_search type has been set. | {'north': -31.456, 'east': 129.653, 'south': -33.234, 'west': 127.453} | No |

The return object will be a list of Simplenamespace objects of the form:
```
namespace(url, type, name)
```

In [ ]:
from auscopecat.api import search
from auscopecat.auscopecat_types import ServiceType

# Search for results that contain the word "magnetite" and have a WMS resource
search_results = search('magnetite', [ServiceType.WMS])
print(f'Total hits: {len(search_results)}')
print(search_results)


### Searching for Records
To search for records, use the `search_records` function in the `api` module.
```
from auscopecat.api import search_records
```
The search_records function allows you to search for more detailed information about records within the catalogue. The call is almost identical to searching for data, only the return objects are different.
```
search_records(pattern: str, ogc_types: list[ServiceType] = None,
               spatial_search_type: SpatialSearchType = None,
               bbox: dict = None) -> list[SimpleNamespace]
```
##### Parameters
| Name | Type | Description | Example | Required |
| ------ | ------ | ------ | ------ | ------ |
| pattern | string | The search string. If there are multiple words, any result that contains any one of the words will be considered a match. To match an exact series of words, use quotation marks, e.g. "Mt Phillips". | 'magnetite' | Yes |
| ogc_types | list[ServiceType] | A list of auscope_types.ServiceType that define OGC service types of interest. Only results containing one or more of these services will be returned. | [ServiceType.WMS, ServiceType.WFS] | No |
| spatial_search | SpatialSearchType | The type of spatial search to be performed with the supplied bounding box. More on spatial searching later. | SpatialSearch.INTERSECTS | No |
| bbox | A dict containing 'north', 'east', 'south' and 'west' values | The bounding box to be used when performing a spatial search. The bounding box will only be applied if a spatial_search type has been set. | {'north': -31.456, 'east': 129.653, 'south': -33.234, 'west': 127.453} | No |

The return object will be a list of Simplenamespace objects of the form:
```
namespace(
    id, name, description, record_info_url,
    constraints[], use_limit_constraints[], access_constraints[], date,
    geographic_elements[
        namespace(type, east, west, north, south)
    ],
    online_resources[
        namespace(url, type, name, description, version)
    ]
)
```


In [ ]:
from auscopecat.api import search_records
from auscopecat.auscopecat_types import ServiceType

# Search for results that contain the phrase "Mt Philips" (note the quotes) and have a WMS resource
search_results = search_records('"Mt Phillips"', [ServiceType.WMS])
if len(search_results) > 0:
    print(f'Total hits: {len(search_results)}')
    print(search_results)
else:
    print('No results returned')


### Searching with Bounds
To search with bounds, a spatial search type and a bounding box must both be provided.
The spatial search types are imported from the auscopecat_types module:
```
from auscopecat.auscopecat_types import SpatialSearchType
```
The spatial search types are:
| Name | Description |
| ------ | ------ |
| SpatialSearchType.INTERSECTS | The results will contain data that intersects in any way with the bounding box. |
| SpatialSearchType.CONTAINS | The results will contain data that reside fully within the supplied bounding box. |
| SpatialSearchType.WITHIN | The results will report bounds that are entirely enclosed by the bounding box. |

In [ ]:
from auscopecat.api import search
from auscopecat.auscopecat_types import SpatialSearchType

# Convenience function to output results
def output_results(results):
    if len(results) > 0:
        print(f'Total hits: {len(results)}')
        print(results)
    else:
        print('No results returned')

query = 'flinders'
bbox = { "north": -32.648, "east": 138.698, "south": -34.157, "west": 137.379 }
# Search without bounds
search_results = search(query, [ServiceType.WFS])
output_results(search_results)
# Search with bounds (this should return a subset of the previous search limited to the specified bounds)
search_results = search(query, [ServiceType.WFS], spatial_search_type=SpatialSearchType.INTERSECTS, bbox=bbox)
output_results(search_results)

## Downloading
To download the data, you can use the `download` function in the `api` module.
```
from auscopecat.api import download
```
You can pass the results of `search` to the `download` function to download returned data, but there are a number of other parameters to help you fine tune results.
```
download(obj: SimpleNamespace, download_type: DownloadType,
         bbox: dict = None, srs_name: str = 'EPSG:4326',
         max_features: int = None, version: str = '1.1.0',
         file_name: str = 'download.csv') -> any
```
##### Parameters
| Name | Type | Description | Example | Required |
| ------ | ------ | ------ | ------ | ------ |
| obj | SimpleNamespace(url, type, name) | The search object, a SimpleNamespace objects with 'url', 'type' and 'name' attributes. | { 'url': 'http://13.211.217.129:8080/geoserver/wfs', 'type': 'WFS', 'name': 'SA:flinders_structures' } | Yes |
| download_type | DownloadType | The type of download. | DownloadType.CSV | Yes |
| bbox | A dict containing 'north', 'east', 'south' and 'west' values | The bounding box for the download data. Only results within the bounds will be returned. | { 'north':-31.456, 'east':129.653, 'south': -33.234, 'west': 127.453 } | Yes |
| srs_name | string | The SRS name. | 'EPGS:4326' | No |
| max_features | int | The maximum number of features to return. | 100 | No |
| version | string | The WFS version. | '1.1.0' | No |
| file_name | string | The file name (including path) for the download. | 'C:/mydir/mydownload.csv' | No |

Calling the method will result in the file being downloaded to your device, barring any errors occurring.

In [ ]:
from auscopecat.api import download
from auscopecat.api import search
from auscopecat.auscopecat_types import DownloadType
from auscopecat.auscopecat_types import ServiceType

# Search for flinders_structures data
query = 'flinders_structures'
bbox = { "north": -32.648, "east": 138.698, "south": -34.157, "west": 137.379 }
search_results = search(query, [ServiceType.WFS])
if len(search_results) > 0:
    print(f'Total hits: { len(search_results) }')
    print(f'Downloading first result: { search_results[0] }')
    # Download the first result
    download(search_results[0], DownloadType.CSV, bbox)
else:
    print('No results returned')